In [ ]:
pip install pdfminer

module import

In [8]:
from pdfminer.pdfinterp import PDFResourceManager,PDFPageInterpreter
from pdfminer.pdfpage import PDFPage
from pdfminer.converter import TextConverter
from pdfminer.layout import LAParams
from io import StringIO
from io import open
import pandas as pd
import numpy as np
from collections import defaultdict
import re 
import requests 
from bs4 import BeautifulSoup 
from tqdm.notebook import tqdm
from datetime import datetime, timedelta

pdf와 hwp를 나누어 저장하기 위해 다운로드 링크를 나누어 저장함. 날짜는 date_list에 저장.

In [9]:
pdf_list = []
hw_list= []
date_list = []
for i in range(6, 33):
    url = 'https://www.bok.or.kr/portal/bbs/B0000245/list.do?menuNo=200761&pageIndex={}'.format(i)
    resp = requests.get(url)
    soup = BeautifulSoup(resp.content, 'lxml')
    total = soup.select('div.fileGoupBox ul li a')
    b= soup.select('span.date')
    count=0
    for idx,a in enumerate(total):
        if (a['href'][1]=='p')&(a['title'][-3:]=='hwp'):
            hw_list.append([b[count].text,a['title'],a['href']])
            count+=1
        elif (a['href'][1]=='p')&(a['title'][-3:]=='pdf'):
            pdf_list.append([b[count-1].text,a['title'],a['href']])
 
print('다운로드 할 pdf 갯수:',len(pdf_list))


다운로드 할 pdf 갯수: 264


In [ ]:
total[0]

<a href="/portal/cmmn/file/fileDown.do?menuNo=200761&amp;atchFileId=KO_00000000000085854&amp;fileSn=1" title="2005년 제17차 금통위 의사록.hwp">
<i class="icon-down"></i> 2005년 제17차 금통위 의사록.hwp
												</a>

In [ ]:
b[0]

<span class="date"><span class="hidden">등록일</span>2005.09.27</span>

In [ ]:
pdf_list[0]

['등록일2017.08.01',
 '제25차 금통위 의사록.pdf',
 '/portal/cmmn/file/fileDown.do?menuNo=200761&atchFileId=KO_00000000000142165&fileSn=1']

 ^ date, filename, download link 의 구조

나누어 저장 !

In [3]:
pdf_list2 = []
title_list2 = []
date_list2 = []

for i in range(len(pdf_list)):
    pdf_list2.append(pdf_list[i][2])
    title_list2.append(pdf_list[i][1])
    date_list2.append(pdf_list[i][0][3:])

데이터 저장

In [4]:
count=0
for i in range(len(pdf_list2)):
    url2 = 'http://www.bok.or.kr' + pdf_list2[i]
    if url2:
        count+=1
    file_res = requests.get(url2)
    with open('{}\n{}'.format(date_list2[i], title_list2[i]), 'wb') as f:
        f.write(file_res.content)

In [5]:
title_date = []
for i in tqdm(range(5, 33)):
    url = 'https://www.bok.or.kr/portal/bbs/B0000245/list.do?menuNo=200761&pageIndex={}'.format(i)

    resp = requests.get(url)
    soup = BeautifulSoup(resp.content, 'lxml')
    date = soup.select('span.titlesub')
    title_date.append(re.findall('\)\((.+?)\)', str(date)))

title_only = []
for i in range(len(title_date)):
    for j in range(len(title_date[0])):
        title_only.append(title_date[i][j])

for i in range(7):
    del(title_only[0])

del(title_only[-8:])

len(title_only)

265

In [6]:
#pdf to txt
def convert_pdf_to_txt(title):
    rsrcmgr = PDFResourceManager()
    retstr = StringIO()
    laparams = LAParams()
    device = TextConverter(rsrcmgr, retstr, codec = 'utf-8', laparams=laparams)
    fp = open('{}'.format(title), 'rb') # 여기에 저장된 url에 대한 response.content를 넣어줘야 함
    interpreter = PDFPageInterpreter(rsrcmgr, device)
    password = ""
    maxpages = 0
    caching = True
    pagenos = set()
    for page in PDFPage.get_pages(fp, pagenos, maxpages=maxpages, password = password, caching = caching, check_extractable=True):
        interpreter.process_page(page)
    text = retstr.getvalue()
    fp.close()
    device.close()
    retstr.close()
    return text

In [7]:
titles = []
for i in range(len(date_list2)):
    titles.append('{}\n{}'.format(date_list2[i], title_list2[i]))

txt_contents = []
for i in range(len(titles)):
    txt_contents.append(convert_pdf_to_txt(titles[i]))

In [14]:
title_only

['2018.2.27.',
 '2018.2.8.',
 '2018.1.18.',
 '2017.12.28.',
 '2017.12.14.',
 '2017.12.7.',
 '2017.11.30.',
 '2017.11.9.',
 '2017.10.19.',
 '2017.9.21.',
 '2017.8.31.',
 '2017.8.10.',
 '2017.7.13.',
 '2017.6.22.',
 '2017.5.25.',
 '2017.4.27.',
 '2017.4.13.',
 '2017.3.23.',
 '2017.2.23.',
 '2017.1.26.',
 '2017.1.13.',
 '2016.12.29.',
 '2016.12.21.',
 '2016.12.15.',
 '2016.11.24.',
 '2016.11.11.',
 '2016.10.27.',
 '2016.10.13.',
 '2016.9.22.',
 '2016.9.9.',
 '2016.8.11.',
 '2016.7.14.',
 '2016.7.1.',
 '2016.6.23.',
 '2016.6.9.',
 '2016.5.13.',
 '2016.4.19.',
 '2016.3.24.',
 '2016.3.10.',
 '2016.2.25.',
 '2016.2.16',
 '2016.1.28.',
 '2016.1.14.',
 '2015.12.24.',
 '2015.12.16.',
 '2015.12.10.',
 '2015.11.26.',
 '2015.11.12.',
 '2015.10.29.',
 '2015.10.15.',
 '2015.9.24.',
 '2015.9.11.',
 '2015.8.27.',
 '2015.8.13.',
 '2015.7.23.',
 '2015.7.9.',
 '2015.6.25.',
 '2015.6.11.',
 '2015.5.15.',
 '2015.4.23.',
 '2015.4.9.',
 '2015.3.26.',
 '2015.3.12.',
 '2015.2.17.',
 '2015.1.29.',
 '2015.1.15.',

In [11]:
titles

['2017.08.01\n제25차 금통위 의사록.pdf',
 '2018.01.16\n제24차 금통위 의사록.pdf',
 '2018.01.02\n제23차 금통위 의사록.pdf',
 '2017.12.26\n제22차 금통위 의사록.pdf',
 '2017.12.19\n제21차 금통위 의사록.pdf',
 '2017.11.07\n제19차 금통위 의사록.pdf',
 '2017.11.07\n제18차 금통위 의사록.pdf',
 '2017.10.10\n제16차 금통위 의사록.pdf',
 '2017.09.19\n제15차 금통위 의사록.pdf',
 '2017.08.29\n제13차 금통위 의사록.pdf',
 '2017.01.10\n제12차 금통위 의사록.pdf',
 '2017.07.07\n제10차 금통위 의사록.pdf',
 '2017.06.13\n제8차 금통위 의사록.pdf',
 '2017.05.02\n제7차 금통위 의사록.pdf',
 '2017.04.07\n제6차 금통위 의사록.pdf',
 '2017.03.14\n제4차 금통위 의사록.pdf',
 '2017.03.14\n제2차 금통위 의사록.pdf',
 '2017.02.14\n제1차 금통위 의사록.pdf',
 '2017.01.31\n제26차 금통위 의사록.pdf',
 '2017.01.17\n제25차 금통위 의사록.pdf',
 '2016.07.19\n제24차 금통위 의사록.pdf',
 '2017.01.03\n제23차 금통위 의사록.pdf',
 '2016.12.13\n제22차 금통위 의사록.pdf',
 '2016.11.15\n제21차 금통위 의사록.pdf',
 '2016.11.15\n제20차 금통위 의사록.pdf',
 '2016.10.11\n제19차 금통위 의사록.pdf',
 '2016.10.11\n제18차 금통위 의사록.pdf',
 '2016.09.27\n제16차 금통위 의사록.pdf',
 '2016.08.02\n제14차 금통위 의사록.pdf',
 '2016.07.19\n제13차 금통위 의사록.pdf',
 '2016.07.12\n제1

In [15]:
dic_data = {'date': titles, 'contents': txt_contents}
index = [i for i in range(len(txt_contents))]

df = pd.DataFrame(dic_data, index = index)
df

,date,contents
0,2017.08.01\n제25차 금통위 의사록.pdf,2017년도 제25차\n\n금융통화위원회(임시) 의사록\n\n한 국 은 행\n\n ...
1,2018.01.16\n제24차 금통위 의사록.pdf,2017년도 제24차\n\n금융통화위원회(정기) 의사록\n\n한 국 은 행\n\n ...
2,2018.01.02\n제23차 금통위 의사록.pdf,2017년도 제23차\n\n금융통화위원회(정기) 의사록\n\n한 국 은 행\n\n ...
3,2017.12.26\n제22차 금통위 의사록.pdf,2017년도 제22차\n\n금융통화위원회(정기) 의사록\n\n한 국 은 행\n\n ...
4,2017.12.19\n제21차 금통위 의사록.pdf,2017년도 제21차\n\n금융통화위원회(정기) 의사록\n\n한 국 은 행\n\n ...
...,...,...
259,2005.10.25\n2005년 제19차 금통위 의사록 - PDF파일.pdf,(cid:4131)(cid:6372)(cid:7458)(cid:7875)(cid:6...
260,2005.09.27\n2005년 제17차 금통위 의사록 - PDF파일.pdf,(cid:4131)(cid:6372)(cid:7458)(cid:7875)(cid:6...
261,2005.08.23\n2005년 제14차 금통위 의사록 - PDF파일.pdf,(cid:4131)(cid:6372)(cid:7458)(cid:7875)(cid:6...
262,2005.08.09\n2005년 제13차 금통위 의사록 - PDF파일.pdf,(cid:4131)(cid:6372)(cid:7458)(cid:7875)(cid:6...


In [19]:
df.to_csv("한국은행의사록corpus.csv")